In [5]:
import pickle
import pandas as pd

course_texts = pickle.load(open('../pickles/course_texts.pkl', 'rb'))

text_df = pd.DataFrame.from_dict(course_texts, orient='index')
text_df.reset_index(inplace=True)
text_df.columns = ['course_id', 'text']
text_df['course_id'] = text_df['course_id'].apply(lambda x: x.split('_')[-1])
text_df

,course_id,text
0,584313,Topic 1: History of\nTopic 2: Chinese Language...
1,584329,Topic 1: Applied Economics\nTopic 2: Mathemati...
2,584381,Topic 1: Artistry\nTopic 2: News Communication...
3,597208,Topic 1: Computer Science and Technology\nDesc...
4,597225,Description: The university computer program w...
...,...,...
3776,2338076,Description: Programme for 2021 [Primary Term ...
3777,2341259,Description: Programme for 2021 [Primary Term ...
3778,2337996,Description: Programme for 2021 [Primary Term ...
3779,1945689,Description: 清华张敏老师带你12 weeks掌握机器学习!8 big clas...


In [3]:
DATASET_NAME = 'MOOCCubeX'
MODEL_NAME = 'NeuMF'
TRAIN_DATASET_FILE = 'train.feather'
TEST_DATASET_FILE = 'test.feather'
MAIN_PATH = f'../data/{DATASET_NAME}/'
TRAIN_DATA_PATH = MAIN_PATH + TRAIN_DATASET_FILE
TEST_DATA_PATH = MAIN_PATH + TEST_DATASET_FILE
MODEL_PATH = f'./models/{DATASET_NAME}/'
MODEL = f'{DATASET_NAME}-{MODEL_NAME}'

item2id = pickle.load(open(MAIN_PATH + 'item2id.pkl', 'rb'))

for item in item2id.items():
    print(item)
    break

train_rating_data = pd.read_feather(TRAIN_DATA_PATH)
test_rating_data = pd.read_feather(TEST_DATA_PATH)
print(train_rating_data.head())

for rating in train_rating_data.iterrows():
    print(rating)
    course_id = 'C_' + str(rating[1]['course_id'])
    if course_id in course_texts.keys():
        print(course_texts[course_id])
        break

# print(course_texts[course_id])
course_id

(375629, 0)
     id  course_id  rating
0  U_24     597214       1
1  U_24     605512       1
2  U_24     597211       1
3  U_24     597314       1
4  U_24     597208       1
(0, id             U_24
course_id    597214
rating            1
Name: 0, dtype: object)
(1, id             U_24
course_id    605512
rating            1
Name: 1, dtype: object)
(2, id             U_24
course_id    597211
rating            1
Name: 2, dtype: object)
(3, id             U_24
course_id    597314
rating            1
Name: 3, dtype: object)
Topic 1: Chinese Language Literature
Description: The current data science and artificial intelligence talent gap is unprecedented, and the need for learning of AI-related knowledge is also significantly increased. The course consists mainly of eight lectures and one conversion challenge task, lectures held once a week.


'C_597314'

In [7]:
import pickle
import torch
tokenizations = pickle.load(open(f'../data/MOOCCubeX/train_embeddings_4_64.pkl', 'rb'))
a = tokenizations[0]
b = tokenizations[1]
torch.cat([a,b]).shape

torch.Size([2, 768])

In [2]:
import numpy as np
import pickle

user = pickle.load(open(f'../data/DBLP_v12/test_users_50.pkl', 'rb'))
user = np.array(user)
user = user.astype(np.int32)
user

In [1]:
import numpy as np
import pickle
item = pickle.load(open(f'../data/DBLP_v12/test_items_50.pkl', 'rb'))
item = np.array(item)
max(item)

2942026

In [42]:
import pickle
papers_text = pickle.load(open("../data/DBLP_v12/papers_text2.pkl", "rb"))
papers_text

{1388: 'Further Results on Independence in Direct-Product Graphs.\nYear: 2000\n#Citations: 1\nJournal\n\nGraph\nDiscrete mathematics\nCombinatorics\nDirect product\nMathematics',
 1688: 'Comparison of GARCH, Neural Network and Support Vector Machine in Financial Time Series Prediction\nYear: 2009\n#Citations: 6\nConference\nSpringer, Berlin, Heidelberg\nThis article applied GARCH model instead AR or ARMA model to compare with the standard BP and SVM in forecasting of the four international including two Asian stock markets indices.These models were evaluated on five performance metrics or criteria. Our experimental results showed the superiority of SVM and GARCH models, compared to the standard BP in forecasting of the four international stock markets indices.\nAutoregressive–moving-average model\nComputer science\nSupport vector machine\nNeural network nn\nArtificial intelligence\nFinancial time series prediction\nAutoregressive conditional heteroskedasticity\nArtificial neural networ

In [32]:
item2id = pickle.load(open("../data/DBLP_v12/item2id.pkl", "rb"))
min(item2id.values()), min(map(lambda x : abs(x), item2id.keys()))

(0, 1388)

In [43]:
import numpy as np
from tqdm import tqdm
import torch

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', '../transformers/BERT/tokenizer/')
model = torch.hub.load('huggingface/pytorch-transformers', 'model', '../transformers/BERT/model/')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

paper_embeddings = np.load("../data/DBLP_v12/paper_embeddings.npy")

for paper, text in tqdm(papers_text.items()):
    model.to(device)
    tokenization = tokenizer(text, max_length=512, padding='max_length', truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        embedding = model(**tokenization).pooler_output.to('cpu')
    if paper in item2id.keys():
        paper_embeddings[item2id[paper]] = embedding
np.save("../data/DBLP_v12/paper_embeddings.npy", paper_embeddings)

Using cache found in /home/rctejon/.cache/torch/hub/huggingface_pytorch-transformers_main
Using cache found in /home/rctejon/.cache/torch/hub/huggingface_pytorch-transformers_main
100%|██████████| 2447040/2447040 [16:12:57<00:00, 41.92it/s]  


In [1]:
import numpy as np
from tqdm import tqdm
import torch
paper_embeddings = np.load("../data/DBLP_v12/paper_embeddings.npy")

paper_embeddings.shape

(2942027, 768)

In [3]:
import pickle

users = pickle.load(open(f'../data/DBLP_v12/train_users_{4}.pkl', 'rb'))
items = pickle.load(open(f'../data/DBLP_v12/train_items_{4}.pkl', 'rb'))
ratings = pickle.load(open(f'../data/DBLP_v12/train_ratings_{4}.pkl', 'rb'))

In [4]:
import numpy as np

users = np.array(users)
items = np.array(items)
ratings = np.array(ratings)

np.save(f'../data/DBLP_v12/train_users_{4}.npy', users)
np.save(f'../data/DBLP_v12/train_items_{4}.npy', items)
np.save(f'../data/DBLP_v12/train_ratings_{4}.npy', ratings)

In [2]:
import numpy as np
users = np.load(f'../data/DBLP_v12/train_users_{4}.npy')
items = np.load(f'../data/DBLP_v12/train_items_{4}.npy')
ratings = np.load(f'../data/DBLP_v12/train_ratings_{4}.npy')